In [1]:
from __future__ import print_function, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
num_epochs = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4
num_classes = 2
echo_step = 3
batch_size = 5
num_batches = total_series_length//batch_size//truncated_backprop_length

In [3]:
def generateData():
    x = np.array(np.random.choice(2, total_series_length, p=[0.5, 0.5]))
    y = np.roll(x, echo_step)
    y[0:echo_step] = 0

    x = x.reshape((batch_size, -1))  # The first index changing slowest, subseries as rows
    y = y.reshape((batch_size, -1))

    return (x, y)

In [5]:
batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

In [6]:
batchX_placeholder

<tf.Tensor 'Placeholder:0' shape=(5, 15) dtype=float32>

In [10]:
init_state = tf.placeholder(tf.float32, [batch_size, state_size])

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)

# Unpack columns
inputs_series = tf.split(batchX_placeholder, truncated_backprop_length, 1)
labels_series = tf.unstack(batchY_placeholder, axis=1)

In [12]:
init_state

<tf.Tensor 'Placeholder_4:0' shape=(5, 4) dtype=float32>

In [14]:
inputs_series

[<tf.Tensor 'split_2:0' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:1' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:2' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:3' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:4' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:5' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:6' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:7' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:8' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:9' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:10' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:11' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:12' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:13' shape=(5, 1) dtype=float32>,
 <tf.Tensor 'split_2:14' shape=(5, 1) dtype=float32>]

In [15]:
labels_series

[<tf.Tensor 'unstack:0' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:1' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:2' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:3' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:4' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:5' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:6' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:7' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:8' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:9' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:10' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:11' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:12' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:13' shape=(5,) dtype=int32>,
 <tf.Tensor 'unstack:14' shape=(5,) dtype=int32>]

In [20]:
# Forward passes
cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
states_series, current_state = tf.nn.static_rnn(cell, inputs_series, init_state)

In [21]:
cell

In [22]:
states_series

[<tf.Tensor 'rnn/basic_rnn_cell/Tanh:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_1:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_2:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_3:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_4:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_5:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_6:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_7:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_8:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_9:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_10:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_11:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_12:0' shape=(5, 4) dtype=float32>,
 <tf.Tensor 'rnn/basic_rnn_cell/Tanh_13:0' shape=(5, 4) dtype=float32>,
 <tf

In [23]:
current_state

<tf.Tensor 'rnn/basic_rnn_cell/Tanh_14:0' shape=(5, 4) dtype=float32>

In [ ]:

logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
predictions_series = [tf.nn.softmax(logits) for logits in logits_series]

losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels) for logits, labels in zip(logits_series,labels_series)]
total_loss = tf.reduce_mean(losses)

train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)

def plot(loss_list, predictions_series, batchX, batchY):
    plt.subplot(2, 3, 1)
    plt.cla()
    plt.plot(loss_list)

    for batch_series_idx in range(5):
        one_hot_output_series = np.array(predictions_series)[:, batch_series_idx, :]
        single_output_series = np.array([(1 if out[0] < 0.5 else 0) for out in one_hot_output_series])

        plt.subplot(2, 3, batch_series_idx + 2)
        plt.cla()
        plt.axis([0, truncated_backprop_length, 0, 2])
        left_offset = range(truncated_backprop_length)
        plt.bar(left_offset, batchX[batch_series_idx, :], width=1, color="blue")
        plt.bar(left_offset, batchY[batch_series_idx, :] * 0.5, width=1, color="red")
        plt.bar(left_offset, single_output_series * 0.3, width=1, color="green")

    plt.draw()
    plt.pause(0.0001)


with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    plt.ion()
    plt.figure()
    plt.show()
    loss_list = []

    for epoch_idx in range(num_epochs):
        x,y = generateData()
        _current_state = np.zeros((batch_size, state_size))

        print("New data, epoch", epoch_idx)

        for batch_idx in range(num_batches):
            start_idx = batch_idx * truncated_backprop_length
            end_idx = start_idx + truncated_backprop_length

            batchX = x[:,start_idx:end_idx]
            batchY = y[:,start_idx:end_idx]

            _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                [total_loss, train_step, current_state, predictions_series],
                feed_dict={
                    batchX_placeholder:batchX,
                    batchY_placeholder:batchY,
                    init_state:_current_state
                })

            loss_list.append(_total_loss)

            if batch_idx%100 == 0:
                print("Step",batch_idx, "Loss", _total_loss)
                plot(loss_list, _predictions_series, batchX, batchY)

plt.ioff()
plt.show()